### Default Setting 

In [20]:
from llama_index import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index.storage.storage_context import StorageContext
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import Document, VectorStoreIndex 
from llama_index.node_parser import SentenceSplitter
import chromadb
import pandas as pd 
import openai
import re
import os
import getpass
import glob 

In [5]:
default_path = os.getcwd()
model_path = os.path.join(default_path, '../../models')
model_dir = os.path.join(model_path, "mistral_origin")
data_path = os.path.join(default_path, '../../../data')
rulebook_path = os.path.join(data_path, 'pdf', 'rules')

In [7]:
file_list = glob.glob(f'{rulebook_path}/*.pdf')
file_list[3]

'/rag/jupyter/llama-index_examples/embedding/../../../data/pdf/rules/메일_화상채팅.pdf'

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [9]:
chroma_client = chromadb.HttpClient(host="192.168.0.146")

In [10]:
chroma_client.list_collections()

[Collection(name=deposit),
 Collection(name=data),
 Collection(name=card),
 Collection(name=loan)]

In [11]:
data_collection = chroma_client.get_or_create_collection("data")

In [12]:
data_collection

Collection(name=data)

In [13]:
data_store = ChromaVectorStore(chroma_collection=data_collection)
data_storage = StorageContext.from_defaults(vector_store=data_store)

In [14]:
page_no = 0

In [102]:
documents = SimpleDirectoryReader(input_files=[file_list[6]]).load_data()

In [103]:
num_mapper = dict({'①':'제1항', '②':'제2항', '③':'제3항', '④':'제4항', '⑤':'제5항', '⑥':'제6항',\
                   '⑦':'제7항', '⑧': '제8항', '⑨': '제9항', '⑩': '제10항', '⑪': '제11항', '⑫': '제12항',\
                  '⑬':'제13항', '⑭':'제14항', '⑮':'제15항', '⑯':'제16항', '⑰':'제17항', '⑱':'제18항', '⑲':'제19항', '⑳':'제20항'})

In [104]:
documents[page_no]

Document(id_='c719b998-e5bf-4e5f-bbae-3d48aeadfa7d', embedding=None, metadata={'page_label': '1', 'file_name': '신여비교통비.pdf', 'file_path': '/rag/jupyter/llama-index_examples/embedding/../../../data/pdf/rules/신여비교통비.pdf', 'file_type': 'application/pdf', 'file_size': 273445, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-06', 'last_accessed_date': '2024-03-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='bb2d5efef6ea765e435421ee434cb404875b4309d1ebd508a124c14f5679eeea', text='1 \n  \n \n \n \n \n \n \n \n \n \n \n여 비 교 통 비 규 정 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n주식회사  핑거 \n \n \n \n \n \n \n \n \n ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {v

In [105]:
def text_cleanse(document):
    '''
    여러 공백 문자열 단일 공백 문자로 변환 
    여러 줄변환 문자 줄변환 문자로 변환 (\n x 2~ => \n x 2) 
    문서 맨 윗 내용이 페이지 번호인 경우 페이지 번호 제거 
    '''
    document.text = re.sub('[^A-Za-z0-9\'\"\-가-힣(){}\\n[]]', '', document.text)
    for num in num_mapper.keys():
        document.text = document.text.replace(num, num_mapper[num])
    document.text = re.sub(' +', ' ', document.text)
    document.text = document.text.strip()
    # document.text = re.sub(r'\r\n{2,}', '\n\n', document.text)
    
    text_list = document.text.splitlines(True)
    if text_list[0].startswith('페이지') or text_list[0].startswith(document.metadata['page_label']):
        text_list = text_list[1:]

    new_text = [] 
    for line in text_list:
        if line == '\n' or line == ' \n':
            continue 
        new_text.append(line) 
    document.text = ''.join(new_text)
    return document

In [106]:
for idx in range(len(documents)):
    documents[idx] = text_cleanse(documents[idx])

In [107]:
print(documents[1].text)

 제1장 총 칙 
제1조 (목적) 
이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해 
국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다. 
제2조 (용어정의 ) 
이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다. 
1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다. 
2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다. 
3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과 
출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다. 
4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다. 
5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다. 
6. “장기출장 ”이라 함은 1개월을 초과하여 수행하는 출장을 말한다. 
7. “장기체재 ”라 함은 장기출장의 일종으로서 동일장소에서 3개월을 초과하여 수행하는 출장을 
말한다. 
제3조 (여비의 계산 기준) 
제1항 여비는 여행 순로에 의하여 산출한다 . 다만, 회사업무의 형편상 또는 천재지변 기타 특별한 
사유로 인하여 순로에 따를 수 없을 경우에는 실지 경과한 노정에 의한다. 
제2항 여비는 아래 각호의 기준으로 계산한다 . 
1. 출발일과 복귀일 포함 (회사의 본사 시간 기준) 
2. 주말, 공휴일 포함 
3. 개인 휴가 사용일 제외 
제4조 (실비지급) 
제1항 특별한 사유로 인하여 본 규정에 규정된 정액 여비로서는 그 실제 비용을 충당할 수 없다고 
인정될 때에는 대표이사 의 승인으로 부족분에 대한 실비를 추가하여 지급할 수 있다. 
제2항 출장과 관련된 업무조사비가 필요한 경우 대표이사의 승인으로 지급할 수 있다. 
제5조 (여비부담 ) 
출장여비는 소속부서 예산에서 부담한다 . 
단, 지원근무를 위한 출장일 경우에는 지원을 받은 소속부서의 예산에서 부담한다 . 
제6조 (지급변경 및

#### Document 재정의 

In [108]:
text = documents[0].text 
documents[0].metadata['page_label']

'1'

In [109]:
import re 

text = '제 1 장 제1장 제 1장 제 2 장 제2장 제 2장' 
# re.findall(r'제.*[0-9].*장', text)
re.findall(r'제.+2.+장', text)

['제 1 장 제1장 제 1장 제 2 장 제2장 제 2장']

In [110]:
text = '제 1 장 제1장 제 1장 제 2 장 제2장 제 2장 제 4장' 
re.findall(r'제 *1 *장', text)

['제 1 장', '제1장', '제 1장']

In [111]:
def get_start_point(documents):
    s_point = 1
    for doc in documents:
        if len(re.findall(r'제 *1 *장', doc.text)) != 0 and (len(re.findall(r'목 *차', doc.text)) == 0 and len(re.findall(r'차 *례', doc.text)) == 0): 
            '''
            print(re.findall(r'제 *1 *장', doc.text))
            print(re.findall(r'목 *차', doc.text))
            print(re.findall(r'차 *례', doc.text))'''
            s_point = doc.metadata['page_label']
            break 
    return int(s_point) - 1

In [112]:
documents[0].metadata['page_label']

'1'

In [113]:
s_point = get_start_point(documents)
s_point

1

In [114]:
print(documents[4].text)

(별표 제1호) 
국내 출장여비 지급기준 
(단위:원) 
구 분 교 통 비 숙박비 
(한도) 일비 식비 
철도임 선박임 항공임 자동차임 
대표이사 KTX특실 
이하 1등객실 
이하 Business 
Class 이하 
고속버스 
우등 
이하 실비 50,000 50,000 
Unit장 
이상 KTX특실 
이하 1등객실 
이하 Economy 
Class 100,000 40,000 20,000 
Part장 KTX일반실 2등객실 Economy 
Class 70,000 30,000 18,000 
Part원 KTX일반실 2등객실 Economy 
Class 50,000 20,000 16,000 
제1항 각 교통수단의 운임은 탑승기준 , 숙박비는 실제 숙박일수 , 일비와 식비는 출장일수에 따라 지급함. 
제2항 2인 이상의 출장의 경우 숙박은 2인 1실을 기준으로 하며, 상위 직위자의 숙박비 한도를 적용함. 
제3항 숙박비와 교통비는 실비 정산하고 , 일비와 식비는 정액 지급함. 
제4항 실비정산 비용은 적격증빙 영수증을 필히 첨부하여야 함. 
제5항 개인차량을 이용하는 경우, 교통비는 아래 별표 제2호에 의한 주유비를 지급하며 , 일비 등을 
지급하므로 출장기간의 운행 기타 비용은 지급하지 아니함. 
(별표 제2호) 
업무상 외출(당일 출장) 지급기준 
구 분 교 통 편 여비지급 방법 비 고 
시내교통편 
가능지역 대 중 교 통 월별 교통비 지원금액 사용 좌석버스 , 전철, 
시내버스 등 
택 시 월별 교통비 지원금액 사용 부득이하게 대중교통 
이용이 어려울 경우 
시내교통편 
불가능지역 대 중 교 통 별표 제1호의 교통비 
개 인 차 량 ◆ 통행료, 주차비 등 실비 정산 
◆ 주유비는 포털싸이트 네이버에서 
검색한 “빠른길 찾기의 최단 
거리”를 적용하여 산출된 
주유비를 왕복 지급한다 . 주유비 지원자 제외


In [115]:
documents[0].metadata

{'page_label': '1',
 'file_name': '신여비교통비.pdf',
 'file_path': '/rag/jupyter/llama-index_examples/embedding/../../../data/pdf/rules/신여비교통비.pdf',
 'file_type': 'application/pdf',
 'file_size': 273445,
 'creation_date': '2024-03-21',
 'last_modified_date': '2024-03-06',
 'last_accessed_date': '2024-03-22'}

In [116]:
def get_doc_content(document):  
    '''
    input: document (페이지 단위로 분할된 document object) 
    output: document_list (장 단위로 분할된 document objects) 
    '''
    doc_list = []; meta_info = dict(); 
    split_spot = re.findall(r'제 *[0-9] *장', document.text))
    splitted_docs = [] 
    for spot in split_spot:
        current_content = spot
        
        splitted_text = document.text.split(spot)
        splitted_docs.extend(

In [126]:
documents[2].text.split('제2장')[1].split('제 3장') #, documents[2].text.split('제2장')[1]

' 국내 출장 여비 \n제9조 (국내출장여비 지급기준) \n국내출장여비지급은 별표 제1호에 정하는 바에 의한다. \n제10조 (국내출장 기간에 따른 여비 조정) \n제1항 출장에 따른 숙박비, 일비, 식비는 출장기간에 따라 아래 각호와 같이 조정하여 지급한다. \n출장기간의 적용은 출장명령서(국내출장신청서 )의 기간에 의한다. 단, 해당 출장자가 이전에 \n동일지역 출장 경력이 있는 경우에는 이전 출장기간을 합산할 수 있다. \n1. 30일차 까지 : 별표 제1호 기준 금액의 100% \n2. 31일차 부터 90일차 까지 : 별표 제1호 기준 금액의 90% \n3. 91일차 이후 : 별표 제1호 기준 금액의 80% \n제2항 장기체재의 경우 주택을 임차하는 것을 원칙으로 하며, 이 때 숙박비는 지급하지 아니한다 . \n주택임차는 담당부서의 주관 하에 출장 당사자의 협조로 출장 개시 1개월 이내에 실시해야 한다. \n제11조 (업무상 외출) \n제1항 업무상 외출(당일 출장)에 대한 여비는 별표 제2호에 의하여 지급한다 . \n제2항 숙박을 불가피하게 해야 할 경우에는 출장 기안으로 전결규정 의 최종결재권자 승인으로 출장 \n여비를 지급할 수 있다. \n제 3 장 국외 출장 여비 \n제12조 (국외출장여비 지급기준 ) \n국외출장 여비지급은 별표 제3호에 정하는 바에 의한다. \n제13조 (국외출장 기간에 따른 여비 조정) \n제1항 출장에 따른 숙박비, 일비, 식비는 출장기간에 따라 아래 각호와 같이 조정하여 지급한다 . \n출장기간의 적용은 출장명령서 (국외출장신청서 )의 기간에 의한다. 단, 해당 출장자가 이전에 \n동일지역 출장 경력이 있는 경우에는 이전 출장기간을 합산할 수 있다. \n1. 30일차 까지 : 별표 제3호 기준 금액의 100% \n2. 31일차 부터 90일차 까지 : 별표 제3호 기준 금액의 90% \n3. 91일차 이후 : 별표 제3호 기준 금액의 80% \n제2항 장기체재의 경우 주택을 임차하는 것을 원칙으로 하며, 이 때 숙박비는 지

In [117]:
doc_list = []

for idx, document in enumerate(documents):
    id_name = document.metadata['file_name'].split('.')[0]
    print(int(document.metadata['page_label']) - 1)
    if int(document.metadata['page_label']) - 1 >= s_point:
        '''
        doc = Document(text=document.text,
                       doc_id=f"{id_name}_doc_{idx}",
                       metadata={"page_label": int(document.metadata['page_label']) - 1, "file_name": document.metadata['file_name']},
                       excluded_llm_metadata_keys = ['page_label', 'file_name']
                )
        doc_list.append(doc)'''
        get_doc_content(document)

0
1
['제1장']
2
['제2장', '제 3 장']
3
['제 4 장']
4
[]
5
[]
6
[]


In [952]:
doc_list[0].text

IndexError: list index out of range

In [953]:
print(doc_list[1].text)

IndexError: list index out of range

In [841]:
from llama_index.node_parser import SentenceSplitter 

In [842]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [843]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name, embed_batch_size=32)

In [844]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [845]:
index = VectorStoreIndex.from_documents(
    doc_list, service_context=service_context, storage_context=data_storage
)

In [846]:
data_collection.count()

188

In [669]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.vector_stores import SimpleVectorStore
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 
from llama_index.postprocessor import SentenceTransformerRerank 
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [671]:
retriever = VectorIndexRetriever(
    index = index,
    service_context=service_context,
    similarity_top_k = 10, 
    verbose=True
)

In [754]:
def get_retrieved_nodes(
    retriever, query_str, vector_top_k=10, similarity_cutoff=0.6, reranker_top_n=3, service_context=None, with_reranker=False
):
    query_bundle = QueryBundle(query_str)   # query bundle 생성 
    retrieved_nodes = retriever.retrieve(query_bundle)   # 유사도가 제일 높은 node 추출 
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)   # 전처리  - 유사 점수 기준 Cutoff 
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    
    if with_reranker:   # 재순위화 
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [715]:
with_reranker = True
cutoff = 0.2

query = '교통비의 정의 기준은 ?' # queries[0]

In [716]:
import time
from llama_index import QueryBundle

start = time.time()
retrieved_nodes = retriever.retrieve(query)
nodes = get_retrieved_nodes(retriever, query, similarity_cutoff=cutoff, service_context=service_context, with_reranker=with_reranker)
end = time.time() - start 
print(end)

1.19443941116333


In [717]:
nodes[0].node.relationships['1'].metadata

{'page_label': 1, 'file_name': '신여비교통비.pdf'}

In [718]:
for key, value in nodes[0].node.relationships.items():
    print(f"key: {key}, value: {value}", end='\n\n')

key: 1, value: node_id='신여비교통비_doc_1' node_type=<ObjectType.DOCUMENT: '4'> metadata={'page_label': 1, 'file_name': '신여비교통비.pdf'} hash='c0395b03779d2b24a5f5c1385ee48dccef329514cfc4293667cf5c8785f1c405'

key: 2, value: node_id='6ab7a5cd-8915-41aa-9ec6-3be743ac8f9e' node_type=<ObjectType.TEXT: '1'> metadata={'page_label': 1, 'file_name': '신여비교통비.pdf'} hash='ca8f23d11a07e5b410cff08e815078185f044ca1d079c82e79c95a9d89f5679f'

key: 3, value: node_id='987143ad-4a77-40cb-a10d-34c7b3715313' node_type=<ObjectType.TEXT: '1'> metadata={} hash='3a63d0dee2494b16d5bf5c31cb32c6e756a5eb06ea396675f703f977fd96b90b'



In [719]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [720]:
print(len(nodes))

3


In [721]:
contexts = [node.text for node in nodes]
scores = [node.score for node in nodes]
contexts, scores 

(['제2항 상위 직책을 대리하거나, 동행하는 경우에는 그 직급의 출장 경비를 지급할 수 있다.\n제7조 (지급의 제한)\n제1항 회사소유 또는 임차한 시설이 있거나 회사가 교통편을 제공하는 경우 이를 이용함을 원칙으로\n한다.\n제2항 장기체재에 따른 숙소 제공 시에는 숙박비를 지원하지 아니한다.\n제3항 장기출장 시에는 야근 식대 및 교통비를 지원하지 아니한다 .',
  '제18조 (장기출장 시 귀향여비 )\n제1항 해외 장기출장 시 국내 귀향에 따른 왕복항공료를 회사에서 분기당 1회씩 지원한다.\n제2항 국내 장기출장 시 귀향에 따른 교통비를 별표 제1호에 의하여 회사에서 월 2회씩 지원한다 .\n단, 거주지를 출장지로 이전한 경우에는 월 1회로 한다.\n부 칙\n제1조 (시행일)\n제1항 이 규정은 2017년 1월 1일부터 제정, 시행한다 .\n제2항 이 규정 시행일 이전의 위 내용과 관련된 규정은 이 규정의 시행 즉시 효력을 상실한다 .\n부 칙\n제1조 (시행일)\n제1항 이 규정은 2018년 8월 1일부터 개정, 시행한다 .',
  '제1장 총 칙\n제1조 (목적)\n이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해\n국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.\n제2조 (용어정의 )\n이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.\n1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.\n2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.\n3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과\n출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.\n4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.\n5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.\n6. “장기출장 ”이라 함은 1개월을 초과하여 수행하

In [724]:
print(nodes[2].text)

제1장 총 칙
제1조 (목적)
이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해
국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.
제2조 (용어정의 )
이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.
1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.
2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.
3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과
출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.
4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.
5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.
6. “장기출장 ”이라 함은 1개월을 초과하여 수행하는 출장을 말한다.
7. “장기체재 ”라 함은 장기출장의 일종으로서 동일장소에서 3개월을 초과하여 수행하는 출장을
말한다.


In [727]:
nodes[0].metadata

{'page_label': 1, 'file_name': '신여비교통비.pdf'}

In [730]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['file_name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [733]:
print(get_info(nodes[0])[2])

제2항 상위 직책을 대리하거나, 동행하는 경우에는 그 직급의 출장 경비를 지급할 수 있다.
제7조 (지급의 제한)
제1항 회사소유 또는 임차한 시설이 있거나 회사가 교통편을 제공하는 경우 이를 이용함을 원칙으로
한다.
제2항 장기체재에 따른 숙소 제공 시에는 숙박비를 지원하지 아니한다.
제3항 장기출장 시에는 야근 식대 및 교통비를 지원하지 아니한다 .


In [753]:
prompt_template = (
    f"""<s>[INST] 질문: {query} \n
    관련 정보: {nodes[2].text} \n 
    관련 정보를 바탕으로 질문에 자연스럽게 답해줘 [/INST] \n"""
)

print(prompt_template)

<s>[INST] 질문: 교통비의 정의 기준은 ? 

    관련 정보: 제1장 총 칙
제1조 (목적)
이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해
국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.
제2조 (용어정의 )
이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.
1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.
2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.
3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과
출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.
4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.
5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.
6. “장기출장 ”이라 함은 1개월을 초과하여 수행하는 출장을 말한다.
7. “장기체재 ”라 함은 장기출장의 일종으로서 동일장소에서 3개월을 초과하여 수행하는 출장을
말한다. 
 
    관련 정보를 바탕으로 질문에 자연스럽게 답해줘 [/INST] 



In [737]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [738]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [744]:
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaTokenizerFast, BitsAndBytesConfig

tokenizer = LlamaTokenizerFast.from_pretrained(os.path.join(model_dir, 'tokenizer'))   # LlamaTokenizer (x)  -> LlamaTokenizerFast (o)
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [748]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [749]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

In [750]:
gened = model.generate(
    **tokenizer(
        prompt_template,
        return_tensors='pt',
        return_token_type_ids=False
    ).to('cuda'),
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # streamer=streamer,
)

In [751]:
result_str = tokenizer.decode(gened[0])

start_tag = f"[/INST]"
start_index = result_str.find(start_tag)

if start_index != -1:
    result_str = result_str[start_index + len(start_tag):].strip()

In [752]:
print(result_str)

질문: 교통비의 정의 기준은? 

답변: 
교통비는 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다. 따라서, 출장 중 이동하는 경로와 수단에 따라 교통비가 지급되는 경우이다.</s>
